# Disassembled thermal circuits from walls description

This notebook presents how the _description of the walls_ is transformed in a set of _disassembled thermal networks_ implemented in the function `wall2TC()` of dm4bem module.

The _description of the walls_ is given in a folder that contains two types of files:
- wall types: thermo-physical properties;
- walls data: regarding the area, temperature & flow-rate sources, coefficients for convection and radiation of the wall surfaces.

The _thermal network_ is a data structure (`dictionary`) containing the matrices A, G, C and the vectors b, f, y.

In [1]:
import pandas as pd
import pd_dm4bem

The input data needed for the thermal model of the building is in a folder.

In [2]:
folder = "bldg_wall2TC"

## Wall types

Let's consider two types of walls with physical properties given in the file `wall_types.csv`.

In [3]:
wall_types = pd.read_csv(folder + '/wall_types.csv')
wall_types

,type,Material,Conductivity,Specific heat,Density,Width,Mesh
0,0,Concrete,1.400,880,2300,0.200,1
1,0,Insulation,0.027,1210,55,0.080,2
2,1,Glass,1.400,750,2500,0.004,1


The file `wall_types.csv` contains:
- the physical properties of the materials given in SI units: __conductivity__ in W·m⁻¹·K⁻¹, __specific heat__ in J·kg⁻¹·K⁻¹, __density__ in kg·m⁻³;
- the __width__ of the layers in m;
- the number of __meshes__ in which each layer is numerically discretized; each mesh has two resistances and a capacity.

There are two types of walls in the file `wall_types.csv` (Figure 1):

- The wall of __type 0__ has two layers: _Concrete_ and _Insulation_. _Concrete_ layer has one mesh (two resistances and a capacity) and _Insulation_ layer has two meshes (i.e., four resistances and two capacities).
- The wall of __type 1__ has one layer: _Glass_, discretized in one mesh.

![wall_types](./bldg_wall2TC/wall_types.svg)
> Figure 1. The two types of wall described in the file `wall_types.csv`

## Walls data
The models of walls are based on the materials given in the file `wall_types.cvs`. For each specific wall, data related to surfaces is given (area, orientation, convection coefficients, radiative properties, temperature and flow-rate sources).

There are three kinds of walls (Table 1):
- generic (e.g., file `walls_generic.csv`), 
- outdoor (e.g., file `walls_out.csv`),
- indoor (e.g., file `walls_in.csv`).

>Table 1. Columns of `walls_generic.csv`, `walls_out.csv`, and `walls_in.csv` files. The columns that are present in the file are maked by ✓ for **_generic**, **_out** and **_in** walls. The columns needed to define the thermal circuit are marked by ✓ in **TC**.

|Column|Unit|Definition                             | Example|_generic  |_out|_in|TC|
|------|----|---------------------------------------|--------|----------|----|---|---|
| ID   |    | Identifiant of each wal               | w2     | ✓        | ✓ | ✓ | ✓ |
| type |    | Type of wall given in `wall_types.csv`| 1      | ✓        | ✓ | ✓ | ✓ |
| Area | m² | Surface area of the wall              | 25     | ✓        | ✓ | ✓ | ✓ |
| β    | °  | Tilt angle: 90° vertical; >90° upward facing| 90 | ✓      | ✓ |   |   |
| γ    | °  | Azimuth: 0° South, 180° North, >0° westward | 90| ✓      | ✓ |   |  |
|albedo|    | Fraction of sunlight that is diffusely reflected by ground surface|0.25| ✓ | ✓ |  | | 
| T0   | °C | Temperature source of outer surface   | To     | ✓        | ✓ |   | ✓ |
| T1   | °C | Temperature source of inner surface   | Ti     | ✓        |   |   | ✓ |
| Q0   | W  | Flow rate source on outer surface     | Qo     | ✓        | ✓ | ✓ | ✓ |
| Q1   | W  | Flow rate source on inner surface     | Qi     | ✓        | ✓ | ✓ | ✓ |
| h0   | W·m⁻²·K⁻¹| Convection coefficient of outer surface | 25 | ✓    | ✓ | ✓ | ✓ |
| h1   | W·m⁻²·K⁻¹| Convection coefficient of inner surface |  8 | ✓    | ✓ | ✓ | ✓ |
| α0   |    | Short-wave absorbtion coefficient of outer surface|0.25|✓ | ✓ | ✓ |   |
| α1   |    | Short-wave absorbtion coefficient of inner surface|0.30|✓ | ✓ | ✓ |   |
| ε0   |    | Long-wave hemispherical emissivity of outer surface|0.85|✓| ✓ | ✓ |   |
| ε1   |    | Long-wave hemispherical emissivity of inner surface|0.70|✓| ✓ | ✓ |   |
| y    |    | Output temperature nodes (by using slicing) |[0, -1]  | ✓ | ✓ | ✓ | ✓ |

__Notes__:
1. The _generic_ walls may define _out_ or _in_ walls by not specifying temperature sources (see `wall_generic.csv`, `wall_out.csv`, and `wall_in.csv` files for examples).

2. The _out_ walls do not need the temperature sources on the inner surfaces, T1.

3. The _in_ walls do not need tilt (β) and azimuth (γ) angles, the albedo of the ground, and the temperature sources on the outer (T0) and the inner (T1) surfaces.

4. The colums __TC__ shows the information needed to obtain the thermal circuit. The data related to radiative exchange is used in the calculation of flow-rate sources in short-wave and for conductances in long-wave radiative exchange.

### Generic wall

Generic walls contain all the information listed in Table 1. For example, `wall_generic.csv` file describes the walls shown in Figure 2.

In [4]:
walls = pd.read_csv(folder + "/walls_generic.csv")
walls

,ID,type,Area,β,γ,albedo,T0,T1,Q0,Q1,h0,h1,α0,α1,ε0,ε1,y
0,w0,1,1,90.0,0.0,0.25,To,NaN,Qo,Qi,20,10,0.25,0.3,0.85,0.7,"[0, -1]"
1,w1,1,1,90.0,0.0,0.25,To,Ti,Qo,Qi,20,10,0.25,0.3,0.85,0.7,1
2,w2,0,10,0.0,90.0,0.25,To,NaN,Qo,Qi,21,11,0.25,0.3,0.85,0.7,"[1, 2, -1]"
3,w3,0,20,NaN,NaN,0.25,NaN,NaN,Qo,Qi,22,12,0.25,0.3,0.85,0.7,"[2, 1, 3]"


The file `walls_generic.csv` describes four walls (Figure 2):
- according to the types of walls:
    - w0 and w1 are of __type__ 1, i.e., concrete and insulation;
    - w2 and w3 are of __type__ 0, i.e., glass.
- according to the temperature sources:
    - w0 and w1 are outdoor walls that have as boundary condition the outdoor temperature __T0__; the temperature on the inner surface is a room temperature, which is an output of the model.
    - w1 is a wall that has temperature sources on both sides.
    - w3 is an inner wall that separates two rooms. It does not need information for solar direct radiation (tilt and azimuth angles and the albedo of the ground surface in front of the wall).

Outdoor walls, such as gw0 (generic wall w0) and gw2 (generic wall w2) in Figure 2, can be obtained from the generic walls by not considering a temperature source __T1__ on inner surface, which is indicated by NaN in rows w0 and w2 for column __T1__.

Likewise, indoor walls, such as gw3 (generic wall 3) in Figure 2, can be obtained from the generic walls by indicating that there are no temperature sources in __T0__ and __T1__, i.e. `T0 = NaN` and `T1 = NaN`.

The outputs __y__ are indicated in a list using [slicing](https://python-reference.readthedocs.io/en/latest/docs/brackets/slicing.html). For example, the outputs of wall w0 are temperature nodes 0 and -1 (i.e., the last node which is node 3) in Figure 2.

The numbering convention of the elements of the circuits is:
- the nodes (i.e., temperatures) are numbered from left to right and from up to down;
- the oriented branches (i.e., flows) are numbered from the node with a lower index to the node with a higher index;
- the __reference__ temperature node is not numbered.

![generic_wall](./bldg_wall2TC/walls_generic.svg)
> Figure 2. Four examples of generic walls described in the file `wall_generic.csv`. Walls gw0 and gw1 are of __type 1__ (glass), described in `wall_type.csv`. Walls gw2 and gw3 are of __type 0__ (concrete and insulation), described in `wall_type.csv`.

The files describing the types (`wall_types.csv`) and the data (`walls_generic.csv`) of the walls contain the information needed to obtain the disassembled thermal circuits. The names of the walls can have a prefix. 

In [5]:
# Thermal circuits from wall types and wall data
TCd_generic = pd_dm4bem.wall2TC(wall_types, walls, prefix="g")

Each matrix (A, G, C) of vector (b, f, y) of the thermal circuit can be accessed as Pandas dataframes.

In [6]:
TCd_generic['gw0']['A']

,gw0_θ0,gw0_θ1,gw0_θ2,gw0_θ3
gw0_q0,1.0,0.0,0.0,0.0
gw0_q1,-1.0,1.0,0.0,0.0
gw0_q2,0.0,-1.0,1.0,0.0
gw0_q3,0.0,0.0,-1.0,1.0


All matrices (A, G, C) and vectors (b, f, y) of each thermal circuit can be printed.

In [7]:
pd_dm4bem.print_TC(TCd_generic['gw0'])

A:
        gw0_θ0  gw0_θ1  gw0_θ2  gw0_θ3
gw0_q0     1.0     0.0     0.0     0.0
gw0_q1    -1.0     1.0     0.0     0.0
gw0_q2     0.0    -1.0     1.0     0.0
gw0_q3     0.0     0.0    -1.0     1.0 

G:
gw0_q0     20.0
gw0_q1    700.0
gw0_q2    700.0
gw0_q3     10.0
dtype: float64 

C:
gw0_θ0       0.0
gw0_θ1    7500.0
gw0_θ2       0.0
gw0_θ3       0.0
dtype: float64 

b:
gw0_q0     To
gw0_q1    0.0
gw0_q2    0.0
gw0_q3    0.0
dtype: object 

f:
gw0_θ0     Qo
gw0_θ1    0.0
gw0_θ2     Qi
gw0_θ3    0.0
dtype: object 

y:
gw0_θ0    1.0
gw0_θ1    0.0
gw0_θ2    0.0
gw0_θ3    1.0
dtype: float64 



All thermal circuits read from the file `walls_generic.csv` can be printed.

In [8]:
print('TCd_generic')
for key in TCd_generic.keys():
    print('Wall:', key)
    pd_dm4bem.print_TC(TCd_generic[key])

TCd_generic
Wall: gw0
A:
        gw0_θ0  gw0_θ1  gw0_θ2  gw0_θ3
gw0_q0     1.0     0.0     0.0     0.0
gw0_q1    -1.0     1.0     0.0     0.0
gw0_q2     0.0    -1.0     1.0     0.0
gw0_q3     0.0     0.0    -1.0     1.0 

G:
gw0_q0     20.0
gw0_q1    700.0
gw0_q2    700.0
gw0_q3     10.0
dtype: float64 

C:
gw0_θ0       0.0
gw0_θ1    7500.0
gw0_θ2       0.0
gw0_θ3       0.0
dtype: float64 

b:
gw0_q0     To
gw0_q1    0.0
gw0_q2    0.0
gw0_q3    0.0
dtype: object 

f:
gw0_θ0     Qo
gw0_θ1    0.0
gw0_θ2     Qi
gw0_θ3    0.0
dtype: object 

y:
gw0_θ0    1.0
gw0_θ1    0.0
gw0_θ2    0.0
gw0_θ3    1.0
dtype: float64 

Wall: gw1
A:
        gw1_θ0  gw1_θ1  gw1_θ2
gw1_q0     1.0     0.0     0.0
gw1_q1    -1.0     1.0     0.0
gw1_q2     0.0    -1.0     1.0
gw1_q3     0.0     0.0    -1.0 

G:
gw1_q0     20.0
gw1_q1    700.0
gw1_q2    700.0
gw1_q3     10.0
dtype: float64 

C:
gw1_θ0       0.0
gw1_θ1    7500.0
gw1_θ2       0.0
dtype: float64 

b:
gw1_q0     To
gw1_q1    0.0
gw1_q2    0.0
gw1_q3    

### Outdoor walls

Outdoor walls do not have temperature sources on indoor surfaces (Figure 3). They can be obtained from generic walls by not indicating the temperature source T1 (on the indoor surface), or they can be described by specific files, such as `walls_in.csv`.

Figure 3a shows a walls w0 and w1 of __type__ 1 (described in `wall_types.csv`) with only one layer which has one mesh.

Figure 3b shows walls w2 and w3 of __type__ 0 (described in `wall_types.csv`) with two layers (concrete and insulation). 

![in_wall](./bldg_wall2TC/walls_out.svg)
> Figure 3. Outdoor walls: a) of __type__ 1 b) of __type__ 0. The __type__ is indicated in the file `wall_types.csv`.

In [9]:
walls = pd.read_csv(folder + '/walls_out.csv')
walls

,ID,type,Area,β,γ,albedo,T0,Q0,Q1,h0,h1,α0,α1,ε0,ε1,y
0,w0,1,1,90,0,0.25,To,Qo,Qi,20,10,0.25,0.3,0.85,0.7,"[0, -1]"
1,w1,1,1,90,0,0.25,To,Qo,Qi,20,10,0.25,0.3,0.85,0.7,1
2,w2,0,10,0,90,0.25,To,Qo,Qi,21,11,0.25,0.3,0.85,0.7,"[1, 2, -1]"
3,w3,0,20,0,90,0.25,Tsoil,Qo,Qi,22,12,0.25,0.3,0.85,0.7,"[2, 1, 3]"


In [10]:
# Thermal circuits from data & type files of walls
TCd_out = pd_dm4bem.wall2TC(wall_types, walls, prefix="o")

# Uncomment below to print all thermal circuits
# print('TCd_out')
# for key in TCd_out.keys():
#    print('Wall:', key)
#    pd_dm4bem.print_TC(TCd_out[key])

### Indoor walls

Indoor walls do not have temperature sources on surfaces (Figure 4). They can be obtained from generic walls by not indicating temperature sources T0 and T1, or they can be described by specific files, such as `walls_in.csv`.

Figure 4a shows walls w0 and w1 of __type__ 1 (described in `wall_types.csv`) with only one layer which has one mesh.

Figure 4b shows wall w2 and w3 of __type__ 0 (described in `wall_types.csv`) with two layers (concrete and insulation); the insulation layer is discretized in two meshes.

![in_wall](./bldg_wall2TC/walls_in.svg)
> Figure 4. Indoor walls: a) of __type__ 1 b) of __type__ 0. The __type__ is indicated in the file `wall_types.csv`.

In [11]:
walls = pd.read_csv(folder + '/walls_in.csv')
walls

,ID,type,Area,Q0,Q1,h0,h1,α0,α1,ε0,ε1,y
0,w0,1,1,Qo,Qi,20,10,0.25,0.3,0.85,0.7,"[0, -1]"
1,w1,1,1,Qo,Qi,20,10,0.25,0.3,0.85,0.7,1
2,w2,0,10,Qo,Qi,21,11,0.25,0.3,0.85,0.7,"[1, 2, -1]"
3,w3,0,20,Qo,Qi,22,12,0.25,0.3,0.85,0.7,"[2, 1, 3]"


In [12]:
# Thermal circuits from data & type files of walls
TCd_in = pd_dm4bem.wall2TC(wall_types, walls, prefix="i")

# Uncomment below to print all thermal circuits
# print('TCd_in')
# for key in TCd_in.keys():
#    print('Wall:', key)
#    pd_dm4bem.print_TC(TCd_in[key])